In [1]:
from sklearn import linear_model
from sklearn import preprocessing 
import pandas as pd
import numpy as np
import datetime,time
import os,sys
import matplotlib.pyplot as plt  
from sklearn.preprocessing import Imputer
from sklearn import svm
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import pickle #pickle模块

D:\Application\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#定义RMSE函数
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [3]:
#读取数据
df = pd.read_csv('05.feature.csv', index_col = 0 )
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(df[['1day_before','2day_before','3day_before']])
df[['1day_before','2day_before','3day_before']]=imp.transform(df[['1day_before','2day_before','3day_before']])
df.to_csv('05.feature.csv')
df_train=df[df['date']<'2017-08-28']
df_test=df[df['date']>='2017-08-28']

In [9]:
#处理特征数据
models_train = list()
models_test = list()
models = list()
corr_list = list()
rmse_list = list()
mae_list= list()
stop_seq = [6,21,22,24,25,26,27,28]
for stop in stop_seq :
    models_train.append(df_train[df_train['up_stop_id'] == stop ])
    models_test.append(df_test[df_test['up_stop_id'] == stop ])    
i = 0 
for  i in range(0,8):
    print("Stop : ", stop_seq[i])  
    #处理训练集
    Y_train = np.array(models_train[i]['num'])
    X_train_conti_attri=np.array([models_train[i]['time_gap'],models_train[i]['dis_61'],models_train[i]['1day_before'],models_train[i]['2day_before'],models_train[i]['3day_before'],]).T
    X_train_sccater_attri=np.array([models_train[i]['week'],models_train[i]['slice_lable']]).T
    #连续属性的处理
    X_train_conti_attri = preprocessing.PolynomialFeatures(3).fit_transform(X_train_conti_attri)#生成特征多项式
    X_train_conti_attri=preprocessing.StandardScaler().fit_transform(X_train_conti_attri)#标准化
    #离散属性的处理
    X_train_sccater_attri=preprocessing.OneHotEncoder().fit_transform(X_train_sccater_attri).toarray()
    #合并特征属性
    X_train=np.concatenate((X_train_conti_attri,X_train_sccater_attri),axis=1)

    #处理测试集
    Y_test = np.array(models_test[i]['num'])
    X_test_conti_attri=np.array([models_test[i]['time_gap'],models_test[i]['dis_61'],models_test[i]['1day_before'],models_test[i]['2day_before'],models_test[i]['3day_before'],]).T
    X_test_sccater_attri=np.array([models_test[i]['week'],models_test[i]['slice_lable']]).T
    #连续属性的处理
    X_test_conti_attri = preprocessing.PolynomialFeatures(3).fit_transform(X_test_conti_attri)#生成特征多项式
    X_test_conti_attri=preprocessing.StandardScaler().fit_transform(X_test_conti_attri)#标准化
    #离散属性的处理
    X_test_sccater_attri=preprocessing.OneHotEncoder().fit_transform(X_test_sccater_attri).toarray()
    #合并特征属性
    X_test=np.concatenate((X_test_conti_attri,X_test_sccater_attri),axis=1)

    #训练集和测试集处理完毕
    #定义参数
    params = {
    'eval_metric':      'rmse',
    'eta':              0.1,
    'max_depth':        7,
    'min_child_weight': 3,
    'subsample':        0.8,
    'colsample_bytree': 0.8,
    'silent':           1,
    'seed':             0
    }
    n_rounds = 300 

    dtrain = xgb.DMatrix( X_train, Y_train )
    dvalid = xgb.DMatrix( X_test, Y_test )

    model = xgb.train( params, dtrain, n_rounds, evals=[(dtrain, 'train'), (dvalid, 'valid')], early_stopping_rounds=20 )
    Y_pred = model.predict( dvalid )
    #计算评价指标
    print('回归系数：',pd.Series(Y_test).reset_index(drop = True).corr(pd.Series(Y_pred)))
    corr_list.append(pd.Series(Y_test).reset_index(drop = True).corr(pd.Series(Y_pred)))
    rmse_list.append(np.sqrt(mean_absolute_error(Y_pred , Y_test)))
    mae_list.append(mean_absolute_error(Y_pred , Y_test))

Stop :  6
[0]	train-rmse:2.21465	valid-rmse:2.01627
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 20 rounds.
[1]	train-rmse:2.07104	valid-rmse:1.89566
[2]	train-rmse:1.9392	valid-rmse:1.79945
[3]	train-rmse:1.82423	valid-rmse:1.72067
[4]	train-rmse:1.72423	valid-rmse:1.65595
[5]	train-rmse:1.62526	valid-rmse:1.57872
[6]	train-rmse:1.54181	valid-rmse:1.5166
[7]	train-rmse:1.46276	valid-rmse:1.47709
[8]	train-rmse:1.39658	valid-rmse:1.44646
[9]	train-rmse:1.33638	valid-rmse:1.40775
[10]	train-rmse:1.28372	valid-rmse:1.43173
[11]	train-rmse:1.23527	valid-rmse:1.42596
[12]	train-rmse:1.19078	valid-rmse:1.42204
[13]	train-rmse:1.15149	valid-rmse:1.41452
[14]	train-rmse:1.11706	valid-rmse:1.40648
[15]	train-rmse:1.08671	valid-rmse:1.39952
[16]	train-rmse:1.05726	valid-rmse:1.39377
[17]	train-rmse:1.02893	valid-rmse:1.39424
[18]	train-rmse:1.00488	valid-rmse:1.38727
[19]	train-rmse:0.985124	valid-rmse:1.39

[109]	train-rmse:0.715524	valid-rmse:3.444
[110]	train-rmse:0.71245	valid-rmse:3.44492
[111]	train-rmse:0.70866	valid-rmse:3.44777
[112]	train-rmse:0.705131	valid-rmse:3.45153
[113]	train-rmse:0.702434	valid-rmse:3.45422
[114]	train-rmse:0.697014	valid-rmse:3.45206


KeyboardInterrupt: 

In [8]:
#输出评价指标
for i in range(0,8):
    print('Stop:',stop_seq[i] )
    print('回归系数：',corr_list[i])
    #print("RMSE:      ", rmse_list[i])
    #print("MAE         ", mae_list[i])

Stop: 6
回归系数： 0.7738981062492423
RMSE:       1.0038038259423243
MAE          1.007622120976448
Stop: 21
回归系数： 0.3312229961041201
RMSE:       0.8729497476928864
MAE          0.7620412619970739
Stop: 22
回归系数： 0.7722474380689117
RMSE:       1.5547557455208734
MAE          2.4172654282301664
Stop: 24
回归系数： 0.36466923849221206
RMSE:       0.7360283931502696
MAE          0.5417377955233679
Stop: 25
回归系数： 0.0748356610373357
RMSE:       0.8906992758764862
MAE          0.7933452000468969
Stop: 26
回归系数： 0.013713293822130802
RMSE:       0.5564282157009965
MAE          0.3096123592281947
Stop: 27
回归系数： 0.2678714216701133
RMSE:       0.8358731740083848
MAE          0.6986839630268514
Stop: 28
回归系数： 0.42692744912782876
RMSE:       0.9952674819250921
MAE          0.9905573605775134
